# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. 

The data source is the Statistiche_giocatori.pkl file, created by the IGNOBEL notebook, which needs to be updated prior any management done in this notebook.

In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list (done)
- Assigning trophies (done)
- Assigning fines (done)
- Calculating total roster value to determine the salary, extra budget and luxury tax (done)
- Update team_real (done)
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC: (done)
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm
- Anything else?

It should be fairly easy to turn this into a dashboard using widgets, but that is for later

In [51]:
from IPython.display import HTML, clear_output
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import progressbar
with open('credential.json','r') as f:
    cred = json.load(f)
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')
import ipywidgets as widgets

In [52]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [56]:
def import_stats():
    stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')
    return stats

## Update personal info
this part is to run through the players in our database and cross check the info with the stats db:
- if the player is not in our db, we don't care: we'll add it when someone wants to buy them
- if the player is in our db but not in stats it means that it left serie A, so the algorithm should change the team to none:
```collection.update_one({'_id': pl['_id']},{'$set':{'info.personal_info.team_real': None}})
```
- if the player has changed teams we need to update the information
- update name if it has changed, based on id

In [57]:
def update_personal_info(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    '''This updates the personal info, name and team, based on the file stats produced in the IGNOBEL notebook.
    the function returns the list of players that are in our db but no longer in FC (e.g. those who retired or left serie a)
    '''
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]

    player_out = []
    posts = collection.find({})
    stats.index = list(stats['Id'])
    for dic in posts:
        Id = dic['_id']
        if Id in stats.index:
            collection.update_one({'_id': Id},{'$set':{'name': stats.loc[Id].Nome }})
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': stats.loc[Id].Squadra}})
        else:
            collection.update_one({'_id': Id},{'$set':{'info.personal_info.team_real': None}})
            player_out.append(collection.find_one({'_id': Id})['name'])
    print('Players no longer in serie A')
    return player_out

In [58]:
#update_personal_info(stats)

## UPDATE stats

In [59]:
#updating the stats of a player based on id
def update_stats(stats, Id, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    '''This updates the stats of the player with the given id, but only the current stats, not the initial quotation
    at the time of market operation, that is recorded when the operation is performed
    '''
    
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    stats.index = list(stats.Id)
    stats_dict = dict(stats[stats['Id'] == int(Id)].T[Id][4:20])

    collection.update_one({'_id': Id}, {'$set': {'info.stats': stats_dict}})

In [60]:
#updating stats of all players in our db
def update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    '''this updates the stats of all the players in our database, if one player is missing it is reported
    
    '''
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    all_pl = list(collection.find({}))
    
    for pl in progressbar.progressbar(all_pl):
        #print(pl['_id'])
        if len(stats[stats['Id'] == pl['_id']]) == 0:
            print(pl['name']+' left serie A')
            #
            continue
        else:
            update_stats(stats, pl['_id'], CR, DB, CO)

In [61]:
# algorithm runs through the entire db and updates stats value
# if player no more in serie a it prints a message, but we need to include change of team_real to none
#it is probably best to 
#update_stats_all(stats)

## Add player to database
First, we create a list with all the players in the stats db who are not in our db

In [62]:
#this function returns the players in stats that are not in our db
def find_missing_ids(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    '''finds the players that are in the stats dataframe but missing from our database'''
    
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    stats.index = list(stats['Id'])
    our_db = collection.find({})
    our_ids = []
    for el in our_db:
        our_ids.append(el['_id'])
    missing_ids = []
    for ind in stats.index:
        if ind in our_ids:
            continue
        else:
            missing_ids.append(ind)
    
    return missing_ids

In [63]:
#this adds a player to our db from the stats db
def add_player(Id, stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    '''adds the player with the given id to the database, so it would give error if the player is already there
    
    '''
    # Dictionary with player current team and loan info
    CurrentTeamDict = {
        'owner': None, #luca, pietro etc
        'squad': None, # 'Main' or 'Primavera'
        'start_date': '',
        'previous_team': None, #last team (e.g. owner, squad)
        'quotation_initial': 0,
        'on_loan': False, #True or False
        'loan_info': None
    }

    # Dictionary with player ownership info
    ContractDict = {
        'owner': None, #this seems redundant
        'start_date': '',
        'cost': 0,
        'acquisition_mode': None, #asta_svincolati, draft, acquisto
        'previous_owner': None, #owner or None
        'quotation_initial': 0
    }

    # Dictionary with player personal info
    PersonalInfoDict = {
        'full_name': '',
        'birthdate': '01/01/1970',
        'nation': '',
        'team_real': '',
        'FC_role': ''
    }

    # Nested dictionary for a single player info
    PlayerDict = {
        '_id': 0,
        'name': '',
        'personal_info': PersonalInfoDict,
        'contract': ContractDict,
        'current_team': CurrentTeamDict,
    }
    transStat = stats.T
    PlayerDict['_id'] = Id
    PlayerDict['name'] = transStat[Id]['Nome']
    PlayerDict['personal_info']['FC_role'] = transStat[Id]['R']
    PlayerDict['personal_info']['team_real'] = transStat[Id]['Squadra']
    PlayerDict['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    PlayerDict['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    PlayerDict['personal_info']['birthdate'] = transStat[Id]['Classe']
    PlayerDict['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    PlayerDict['contract']['owner'] = None
    PlayerDict['current_team']['owner'] = None
    PlayerDict['current_team']['quotation_initial'] = transStat[Id]['Qt. I'] 
    temp = dict(transStat[Id][4:20])
    temp['Qt_I'] = temp.pop('Qt. I')
    temp['Qt_A'] = temp.pop('Qt. A')
    PlayerDict['stats'] = temp
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    collection.insert_one(PlayerDict)
    
    return PlayerDict

In [64]:
#this function adds all the players in the missing_ids list.

In [65]:
def add_multi_pl(Ids, stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    '''adds all the players in the list of Ids to our database'''
    pls = []
    for Id in Ids:
        pl = add_player(Id, stats, CR, DB, CO)
        pls.append(pl['name'])
    return pls

In [66]:
#missing_ids = find_missing_ids(stats)
#missing_ids 

#The first round returned these four players, which now are in our db, so running the missing_id function returns an empty list
#add_multi_pl(missing_ids, stats)

## Manage roster value and extra budget/luxury tax

This class acts directly on the full database, but we could in principle create a session class, like we did for the market operations, and then close the session by copying everything only at the end, but since the db has only 8 entries I don't see the point, it is easy to check and fix stuff

In [69]:
class manager:
    '''class manager is designed to perform changes to the managers database, based on all possibilities'''
    
    def __init__(self, owner, stats = stats, CR = cred['cred'], DB = 'Game', CO = 'Managers', CP = 'Players'):
        
        self.par = CR, DB, CO
        cluster = MongoClient(CR)
        db = cluster[DB]
        self.collection = db[CO]
        self.coll_P = db[CP]
        
        self.dic = self.collection.find_one({'owner': owner})
        self.dic_p = self.coll_P.find(({'info.contract.owner': owner,'info.current_team.squad':'main'}))
        
        pl_value = 0
        for pl in self.dic_p:
            pl_value += pl['info']['stats']['Qt_A']
        
        self.main_roster_value = pl_value
        
        self.owner = self.dic['owner']
        self.team_name = self.dic['team_name']
        self.budget = self.dic['budget']
        self.palmares = self.dic['palmares']
        self.total_wins = self.dic['total_wins']
    
    def modify_budget(self, delta):
        '''add delta to the budget of the owner method (delta can be negative)'''
        
        self.collection.update_one({'owner': self.owner},{'$inc':{'budget':delta}})
        self.budget = self.budget + delta #otherwise you have to re-run the cell to see the modification
    
    def assign_trophy(self, trophy, season, prize_money):
        '''assigns a specific trophy, need to specifu name, prize and season'''
        
        
        dic = { 
            'Type': trophy,
            'Prize_eur': prize_money,
            'Season': season
              }
        
        self.collection.update_one({'owner': self.owner},{'$addToSet': {'palmares': dic}})
        self.collection.update_one({'owner': self.owner},{'$inc': {'total_wins': prize_money}})
        
        self.palmares.append(dic)
        self.total_wins += prize_money
    
    def assign_fine(self, fine = 4, reason = 'Missing lineup'):
        '''assigns fine: default is the missing lineup, but can be used for luxury tax'''
        d = date.today()
        
        dic = { 
            'Reason': reason,
            'Fine_eur': fine,
            'date': d.strftime('%Y/%m/%d')
              }
        self.collection.update_one({'owner': self.owner},{'$addToSet': {'fines': dic}})
    
    def assign_extra_budget(self, extra_budget = 100, salary_cap = 350):
        '''to be used only at the end of the season, when determining the starting budget for the next season'''
        
        temp = extra_budget + salary_cap - self.main_roster_value
        
        extra = max(0, temp)
        self.collection.update_one({'owner': self.owner},{'$inc': {'budget': extra}})
        
        if temp < 0:
            print(self.owner+ ' exceeded the luxury tax limit. He will paying '+ str(-temp)+ ' EUR to the league.')
        else:
            print(self.owner+ ' was assigned '+str(extra)+ ' FM.')

In [40]:
luca = manager('luca')

In [41]:
#THIS assigns extra budget with the formula: Extra budget = 100 + salary_cap - roster_value (see method definition)
#luca.assign_extra_budget()

In [42]:
enzo = manager('enzo')

In [43]:
#Assign prize (already done, so don't repeat otherwise there are duplicates in the db)
#enzo.assign_trophy('Coppa di Lega', '20/21', 25)

In [44]:
nanni = manager('nanni')
mario = manager('mario')

#Fines have been already assigned, so do not run again
#nanni.assign_fine()
#mario.assign_fine()

# Control Dashboard

In [68]:
btn_update_stats = widgets.Button(description='Update Players Stats',layout=widgets.Layout(width='200px', height='40px'))

def update_stats_event(obj):
    stats = import_stats()
    update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players')
    
btn_update_stats.on_click(update_stats_event)
btn_update_stats.style.button_color = 'skyblue'

btn_personal_info= widgets.Button(description='Update Players Info',layout=widgets.Layout(width='200px', height='40px'))

def update_info_event(obj):
    stats = import_stats()
    update_personal_info(stats, CR = cred['cred'], DB = 'Game', CO = 'Players')
    
btn_personal_info.on_click(update_info_event)
btn_personal_info.style.button_color = 'skyblue'

btn_missing_players= widgets.Button(description='Add New Players',layout=widgets.Layout(width='200px', height='40px'))

def new_players_event(obj):
    stats = import_stats()
    missing_ids = find_missing_ids(stats)
    add_multi_pl(missing_ids, stats)
    
btn_missing_players.on_click(new_players_event)
btn_missing_players.style.button_color = 'skyblue'



#display(btn_personal_info)
#display(btn_update_stats)
#display(btn_missing_players)

box_buttons = widgets.VBox([btn_personal_info, btn_update_stats, btn_missing_players])

display(box_buttons)

 95% (542 of 566) |##################### | Elapsed Time: 0:02:03 ETA:   0:00:04

BADU left serie A


 97% (554 of 566) |##################### | Elapsed Time: 0:02:06 ETA:   0:00:02

DIALLO left serie A


 98% (559 of 566) |##################### | Elapsed Time: 0:02:06 ETA:   0:00:01

MILLICO left serie A


100% (566 of 566) |######################| Elapsed Time: 0:02:08 Time:  0:02:08


## Managers db 
I am commenting out the actual command inside the widget to prevent accidental changes, uncomment before use

In [46]:
def mod_bud_event():
    budget_num = widgets.BoundedFloatText(min=-1000, max=1000, value=0, step=1, description = 'Budget Change')

    def budget_num_eventhandler(change):
        global budget_delta
        budget_delta = change.new

    budget_num.observe(budget_num_eventhandler, names='value')

    btn_conf_budget = widgets.Button(description='Confirm Budget Change')

    def conf_budget_eventhandler(obj):
        clear_output()
        display(manager_selection_group)
        manag.modify_budget(budget_delta)
        print(manag.owner+'\'s new Budget: '+str(manag.budget))


    btn_conf_budget.on_click(conf_budget_eventhandler)

    #display(budget_num)
    #display(btn_conf_budget)
    budget_box = widgets.HBox([budget_num, btn_conf_budget])
    display(budget_box)

In [47]:
def ass_trophy_event():
    trophies = ['Choose one', 'Scudetto', 'Coppa di Lega', 'Champions', 'Supercoppa', 'Porta Violata', 'Cartellino Facile', 'Panchina D\'Oro', 'Caduti']

    trophies_menu = widgets.Dropdown(options=trophies, description = 'Trophy')

    def assign_trophy_eventhandler(change):
        if change.new == 'Choose one':
            None
        else:
            global trophy
            trophy = change.new


    trophies_menu.observe(assign_trophy_eventhandler, names = 'value')

    #display(trophies_menu)

    season_text= widgets.Text(
        placeholder='YY/YY+1',
        description='Season',
        disabled=False
    )


    def season_eventhandler(change):
        global season
        season = change.new

    season_text.observe(season_eventhandler, names='value')

    #display(season_text)

    prize_num = widgets.BoundedFloatText(min=0, max=1000, value=0, step=1, description = 'Prize (EUR)')

    def prize_num_eventhandler(change):
        global prize_money
        prize_money = change.new

    prize_num.observe(prize_num_eventhandler, names='value')

    btn_conf_trophy = widgets.Button(description='Confirm Trophy')

    def conf_trophy_eventhandler(obj):
        clear_output()
        display(manager_selection_group)
        #manag.assign_trophy(trophy, season, prize_money)
        print('The trophy '+trophy+ ' was assigned to '+ manag.owner)


    btn_conf_trophy.on_click(conf_trophy_eventhandler)


    trophy_par = widgets.VBox([trophies_menu, season_text, prize_num])
    trophy_group = widgets.HBox([trophy_par, btn_conf_trophy])

    display(trophy_group)

In [48]:
def ass_fine_event():
    reasons = ['Choose one', 'Missing lineup', 'Luxury Tax']

    reasons_menu = widgets.Dropdown(options=reasons, description = 'Reason')

    def reasons_eventhandler(change):
        if change.new == 'Choose one':
            None
        else:
            global reason
            reason = change.new


    reasons_menu.observe(reasons_eventhandler, names = 'value')


    #display(season_text)

    fine_num = widgets.BoundedFloatText(min=0, max=1000, value=0, step=1, description = 'Amount (EUR)')

    def fine_num_eventhandler(change):
        global fine
        fine = change.new

    fine_num.observe(fine_num_eventhandler, names='value')

    btn_conf_fine = widgets.Button(description='Confirm Fine')

    def conf_fine_eventhandler(obj):
        clear_output()
        display(manager_selection_group)
        #manag.assign_fine(fine, reason)
        print('A fine of the amount of '+str(fine)+ ' was assigned to '+ manag.owner+' for the following reason: '+ reason)


    btn_conf_fine.on_click(conf_fine_eventhandler)


    fine_par = widgets.VBox([reasons_menu, fine_num])
    trophy_group = widgets.HBox([fine_par, btn_conf_fine])

    display(trophy_group)

In [49]:
owners = ['Choose one', 'luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']

own_menu = widgets.Dropdown(options=owners, description = 'Manager')
owner = 'Choose one'

def select_owner_eventhandler(change):
    if change.new == 'Choose one':
        None
    else:
        global manag
        global owner
        owner = change.new
        stats = import_stats()
        manag = manager(change.new, stats = stats, CR = cred['cred'])
    

own_menu.observe(select_owner_eventhandler, names = 'value')

btn_modify_budget = widgets.Button(description='Modify Budget')

def mod_budget_eventhandler(obj):
    if owner == 'Choose one':
        None
    else:
        mod_bud_event()

btn_modify_budget.on_click(mod_budget_eventhandler)

btn_assign_trophy = widgets.Button(description='Assign Trophy')

def ass_trophy_eventhandler(obj):
    if owner == 'Choose one':
        None
    else:
        ass_trophy_event()
    

btn_assign_trophy.on_click(ass_trophy_eventhandler)

btn_assign_fine = widgets.Button(description='Assign Fine')

def ass_fine_eventhandler(obj):
    if owner == 'Choose one':
        None
    else:
        ass_fine_event()
    

btn_assign_fine.on_click(ass_fine_eventhandler)

#display(own_menu)
#display(btn_confirm_manager)

manager_selection_group = widgets.HBox([own_menu,btn_modify_budget,btn_assign_trophy, btn_assign_fine])
display(manager_selection_group)

## Assign extra budget to all managers

In [50]:
ex_bud_num = widgets.BoundedFloatText(min=0, max=1000, value=0, step=1, description = 'Extra Budget')

def ex_bud_eventhandler(change):
    global extra_budget
    extra_budget = change.new

ex_bud_num.observe(ex_bud_eventhandler, names='value')

sal_cap_num = widgets.BoundedFloatText(min=0, max=1000, value=0, step=1, description = 'Salary Cap')

def sal_cap_eventhandler(change):
    global salary_cap
    salary_cap = change.new

sal_cap_num.observe(sal_cap_eventhandler, names='value')

btn_assign_extra = widgets.Button(description='Assign to All')

def ass_extra_eventhandler(obj):
    owners_ex = ['luca','pietro','enzo','nanni','mario','emiliano','franky','musci8']
    for owner_ex in owners_ex:
        man_ex = manager(owner_ex, stats = stats, CR = cred['cred'])
        man_ex.assign_extra_budget(extra_budget, salary_cap)

btn_assign_extra.on_click(ass_extra_eventhandler)

v_box = widgets.VBox([ex_bud_num,sal_cap_num])
h_box = widgets.HBox([v_box,btn_assign_extra])
    
display(h_box)